# 0. Overview

## 0.1 Versions
 - DSUNet101<br>
 Baseline Model for Deep Supervised Learning<br>
 https://github.com/SeuTao/TGS-Salt-Identification-Challenge-2018-_4th_place_solution/blob/master/model/model.py<br>
 https://github.com/SeuTao/TGS-Salt-Identification-Challenge-2018-_4th_place_solution/blob/master/train.py<br>
 CV Score : , LB Score : 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os, cv2, time, random, warnings
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import ttach as tta

import albumentations as albu
import segmentation_models_pytorch as smp

warnings.filterwarnings('ignore')

In [ ]:
N_SPLITS = 8
SEED = 1234
FOLD = 0
VERSION = f'DSUNet101_{FOLD}'
PATH = '../input'
TRAIN_PATH = '../input/train_images_525/train_images_525'
TEST_PATH = '../input/test_images_525/test_images_525' 
TRAIN_MASK_PATH = '../input/train_masks_525/train_masks_525'
WEIGHT_PATH = '../input/weights'
SUBMISSION_PATH = '../input/submissions'
N_JOBS = 4

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_on_gpu = torch.cuda.is_available()

IMAGE_SIZE = (1400, 2100)
IMAGE_SIZE_525 = (350, 525)

In [ ]:
def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# 1. Preparation

## 1.1 Load Datasets

In [ ]:
class_names_dict = {'Fish':1, 'Flower':2, 'Gravel':3, 'Sugar':4}
train_df = pd.read_csv('../input/train.csv')
train_df['ImageId'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
train_df['Class'] = train_df['Image_Label'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()
train_df['class_id'] = train_df['Class'].map(class_names_dict)

print(train_df.shape)
train_df.head()

In [ ]:
sub_df = pd.read_csv('../input/sample_submission.csv')
sub_df['ImageId'] = sub_df['Image_Label'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])

## 1.2 Split Data

In [ ]:
def rle_decode(mask_rle: str = '', shape: tuple = (1400, 2100)):
    '''
    Decode rle encoded mask.
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

def make_mask_count(df: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (1400, 2100)):
    """
    Create mask based on df, image name and shape.
    """
    encoded_masks = df
    masks = np.zeros((shape[0], shape[1]), dtype=np.float32)
    if encoded_masks is np.nan:
        masks = 0
    else:
        mask = rle_decode(encoded_masks)
        masks[:, :] = mask
        masks = np.sum(masks==1)
    return masks

train_df['pixel_count'] = train_df["EncodedPixels"].apply(make_mask_count)
train_df['pixel_group'] = train_df['pixel_count'].apply(lambda x: np.ceil(x/500000))
train_df.head(2)

In [ ]:
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()[['ImageId','pixel_group']]
mask_count_df.sort_values('pixel_group', ascending=False, inplace=True)
print(mask_count_df.shape)
mask_count_df.head()

In [ ]:
ids = mask_count_df.index.values

li = [
    [train_index, test_index]
    for train_index, test_index in StratifiedKFold(
        n_splits=N_SPLITS, shuffle=True, random_state=SEED
    ).split(ids, mask_count_df["pixel_group"])
]

train_idx, val_idx = ids[li[FOLD][0]], ids[li[FOLD][1]]
train_ids = np.array(mask_count_df['ImageId'].iloc[train_idx])
val_ids = np.array(mask_count_df['ImageId'].iloc[val_idx])
test_ids = sub_df["Image_Label"].apply(lambda x: x.split("_")[0]).drop_duplicates().values

print(f"train_length {len(train_idx)}")
print(f"valid_length {len(val_idx)}")

## 1.3 Loss Function

In [ ]:
def dice_coef_np(y_pred, y_true, smooth=1e-9, activation=False):
    if activation:
        y_pred = 1/(1 + np.exp(-y_pred))### sigmoid
    y_pred = np.asarray(y_pred).astype(np.bool)
    y_true = np.asarray(y_true).astype(np.bool)
    if y_pred.shape != y_true.shape:
        raise ValueError("Shape mismatch: y_pred and y_true must have the same shape.")
    # Compute Dice coefficient
    intersection = np.logical_and(y_pred, y_true)
    return (2. * intersection.sum() + smooth) / (y_pred.sum() + y_true.sum() + smooth)

In [ ]:
def dice_no_threshold(
    outputs: torch.Tensor,
    targets: torch.Tensor,
    eps: float = 1e-7,
    threshold: float = None,
    activation: bool = False
):
    if activation:
        outputs = torch.sigmoid(outputs)

    if threshold is not None:
        outputs = (outputs > threshold).float()

    intersection = torch.sum(targets * outputs)
    union = torch.sum(targets) + torch.sum(outputs)
    dice = 2 * intersection / (union + eps)

    return dice

def f_score(pr, gt, beta=1, eps=1e-7, threshold=None, activation='sigmoid'):
    """
    Args:
        pr (torch.Tensor): A list of predicted elements
        gt (torch.Tensor):  A list of elements that are to be predicted
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: IoU (Jaccard) score
    """

    if activation is None or activation == "none":
        activation_fn = lambda x: x
    elif activation == "sigmoid":
        activation_fn = torch.nn.Sigmoid()
    elif activation == "softmax2d":
        activation_fn = torch.nn.Softmax2d()
    else:
        raise NotImplementedError(
            "Activation implemented for sigmoid and softmax2d"
        )

    pr = activation_fn(pr)

    if threshold is not None:
        pr = (pr > threshold).float()

    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp
    fn = torch.sum(gt) - tp

    score = ((1 + beta ** 2) * tp + eps) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + eps)

    return score

class DiceLoss(nn.Module):
    __name__ = 'dice_loss'

    def __init__(self, eps=1e-7, activation='sigmoid', beta=1.):
        super().__init__()
        self.activation = activation
        self.eps = eps
        self.beta = beta

    def forward(self, y_pr, y_gt):
        return 1 - f_score(y_pr, y_gt, beta=self.beta, 
                           eps=self.eps, threshold=None, 
                           activation=self.activation)


class BCEDiceLoss(DiceLoss):
    __name__ = 'bce_dice_loss'

    def __init__(self, eps=1e-7, activation='sigmoid', beta=1., lambda_dice=1.0, lambda_bce=1.0):
        super().__init__(eps, activation)
        if activation == None:
            self.bce = nn.BCELoss(reduction='mean')
        else:
            self.bce = nn.BCEWithLogitsLoss(reduction='mean')
        self.lambda_dice=lambda_dice
        self.lambda_bce=lambda_bce

    def forward(self, y_pr, y_gt):
        dice = super().forward(y_pr, y_gt)
        bce = self.bce(y_pr, y_gt)
        return (self.lambda_dice*dice) + (self.lambda_bce* bce)

## 1.4 Optimizer

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-2, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
            
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)
        

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)
        
    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

## 1.5 Data Generator

In [ ]:
def make_mask(df: pd.DataFrame, image_name: str = "img.jpg", shape: tuple = IMAGE_SIZE_525):
    """
    Create mask based on df, image name and shape.
    """
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)
    df = df[df["ImageId"] == image_name]
    for idx, im_name in enumerate(df["ImageId"].values):
        for classidx, classid in enumerate(["Fish", "Flower", "Gravel", "Sugar"]):
            mask = cv2.imread(f"{PATH}/{TRAIN_MASK_PATH}/"
                + classid
                + im_name
            )
            if mask is None:
                continue
            if mask[:, :, 0].shape != (350, 525):
                mask = cv2.resize(mask, (525, 350))
            masks[:, :, classidx] = mask[:, :, 0]
    masks = masks / 255
    return masks


In [ ]:
def get_training_augmentation():
    train_transform = [
        albu.Resize(320, 480),
        albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),
        albu.Rotate(limit=20,p=0.5),
        albu.GaussNoise(p=0.2),        
        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=0.3, border_mode=0),
        albu.GridDistortion(p=0.3),
        albu.OpticalDistortion(p=0.2, distort_limit=2, shift_limit=0.5),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.Resize(320, 480),
        albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#         albu.HorizontalFlip(p=0.5),
    ]
    return albu.Compose(test_transform)

def get_test_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.Resize(320, 480),
        albu.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]
    return albu.Compose(test_transform)

In [ ]:
# Dataset class
class CloudDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame = None,
        datatype: str = "train",
        img_ids: np.array = None,
        transforms = albu.Compose([albu.Resize(320, 480)]),
    ):
        self.df = df
        if datatype != "test":
            self.data_folder = TRAIN_PATH
        else:
            self.data_folder = TEST_PATH
        self.img_ids = img_ids
        self.transforms = transforms

    def __getitem__(self, idx):
        
        image_name = self.img_ids[idx]
        mask = make_mask(self.df, image_name)
        image_path = os.path.join(self.data_folder, image_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        augmented = self.transforms(image=img, mask=mask)
        img = np.transpose(augmented["image"], [2, 0, 1])
        mask = np.transpose(augmented["mask"], [2, 0, 1])
        
        is_empty = np.array([0., 0., 0., 0.])
        for i in range(4):
            if np.sum(mask[i])==0:
                is_empty[i] = 1.
                
        return img, mask, is_empty

    def __len__(self):
        return len(self.img_ids)

# 2. Modeling

## 2.1 Model Definition

In [ ]:
num_workers = N_JOBS
batch_size = 4

train_dataset = CloudDataset(df=train_df,
                             datatype="train",
                             img_ids=train_ids,
                             transforms=get_training_augmentation(),)

valid_dataset = CloudDataset(df=train_df,
                             datatype="valid",
                             img_ids=val_ids,
                             transforms=get_validation_augmentation(),)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size,
                          num_workers=num_workers)

valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=1, #batch_size
                          num_workers=num_workers)

In [ ]:
class Decoder(nn.Module):
    def __init__(self,in_channels, channels, out_channels):
        super(Decoder, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels, channels, kernel_size=3,padding=1),
                                   nn.BatchNorm2d(channels),
                                   nn.ReLU(inplace=True))
        self.conv2 = nn.Sequential(nn.Conv2d(channels, out_channels, kernel_size=3,padding=1),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True))
        self.SCSE = SCSEBlock(out_channels)

    def forward(self, x, e = None):
        x = F.upsample(x, scale_factor=2, mode='bilinear')
        if e is not None:
            x = torch.cat([x,e],1)
            x = F.dropout2d(x, p = 0.50)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.SCSE(x)
        return x
    
class SCSEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SCSEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)

        self.channel_excitation = nn.Sequential(nn.Linear(channel, int(channel//reduction)),
                                                nn.ReLU(inplace=True),
                                                nn.Linear(int(channel//reduction), channel),
                                                nn.Sigmoid())

        self.spatial_se = nn.Sequential(nn.Conv2d(channel, 1, kernel_size=1,
                                                  stride=1, padding=0, bias=False),
                                        nn.Sigmoid())

    def forward(self, x):
        bahs, chs, _, _ = x.size()

        # Returns a new tensor with the same data as the self tensor but of a different size.
        chn_se = self.avg_pool(x).view(bahs, chs)
        chn_se = self.channel_excitation(chn_se).view(bahs, chs, 1, 1)
        chn_se = torch.mul(x, chn_se)

        spa_se = self.spatial_se(x)
        spa_se = torch.mul(x, spa_se)
        return torch.add(chn_se, 1, spa_se)
    
class model34_DeepSupervion(nn.Module):
    def __init__(self, num_classes=1, image_size=(320, 480), mask_class = 2):
        super(model34_DeepSupervion, self).__init__()

        self.num_classes = num_classes
        self.image_size = image_size

        self.encoder = torchvision.models.resnet34(pretrained=True)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Sequential(self.encoder.conv1,
                                   self.encoder.bn1,
                                   self.encoder.relu)

        self.conv2 = self.encoder.layer1
        self.conv3 = self.encoder.layer2
        self.conv4 = self.encoder.layer3
        self.conv5 = self.encoder.layer4

        self.center_global_pool = nn.AdaptiveAvgPool2d([1,1])
        self.center_conv1x1 = nn.Conv2d(512, 64, kernel_size=1)
        self.center_fc = nn.Sequential(nn.Linear(64, mask_class), 
                                       nn.Sigmoid())

        self.center = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3,padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(512, 256, kernel_size=3, padding=1),
                                    nn.BatchNorm2d(256),
                                    nn.ReLU(inplace=True),
                                    nn.MaxPool2d(kernel_size=2,stride=2))

        self.decoder5 = Decoder(256 + 512, 512, 64)
        self.decoder4 = Decoder(64 + 256, 256, 64)
        self.decoder3 = Decoder(64 + 128, 128, 64)
        self.decoder2 = Decoder(64 + 64, 64, 64)
        self.decoder1 = Decoder(64, 32, 64)

        self.logits_no_empty = nn.Sequential(nn.Conv2d(320, 64, kernel_size=3, padding=1),
                                             nn.ReLU(inplace=True),
                                             nn.Conv2d(64, self.num_classes, kernel_size=1, padding=0), 
                                             nn.Sigmoid())

        self.logits_final = nn.Sequential(nn.Conv2d(320+64, 64, kernel_size=3, padding=1),
                                          nn.ReLU(inplace=True),
                                          nn.Conv2d(64, self.num_classes, kernel_size=1, padding=0), 
                                          nn.Sigmoid())

    def forward(self, x):
        conv1 = self.conv1(x)     #1/4
        conv2 = self.conv2(conv1) #1/4
        conv3 = self.conv3(conv2) #1/8
        conv4 = self.conv4(conv3) #1/16
        conv5 = self.conv5(conv4) #1/32

        center_512 = self.center_global_pool(conv5)
        center_64 = self.center_conv1x1(center_512)
        center_64_flatten = center_64.view(center_64.size(0), -1)
        center_fc = self.center_fc(center_64_flatten)

        f = self.center(conv5)
        d5 = self.decoder5(f, conv5)
        d4 = self.decoder4(d5, conv4)
        d3 = self.decoder3(d4, conv3)
        d2 = self.decoder2(d3, conv2)
        d1 = self.decoder1(d2)

        hypercol = torch.cat((
            d1,
            F.upsample(d2, scale_factor=2,mode='bilinear'),
            F.upsample(d3, scale_factor=4, mode='bilinear'),
            F.upsample(d4, scale_factor=8, mode='bilinear'),
            F.upsample(d5, scale_factor=16, mode='bilinear')),1)
        hypercol = F.dropout2d(hypercol, p = 0.50)

        x_no_empty = self.logits_no_empty(hypercol)
        hypercol_add_center = torch.cat((
            hypercol,
#             F.upsample(center_64, scale_factor=128,mode='bilinear')),1)
            F.upsample(center_64, size=self.image_size, mode='bilinear')),1)

        x_final = self.logits_final(hypercol_add_center)
        return center_fc, x_no_empty, x_final

In [ ]:
model = model34_DeepSupervion(num_classes=4, image_size=(320, 480), mask_class = 4)

if train_on_gpu:
    model.to(DEVICE)
    
model

In [ ]:
### todo
criterion = BCEDiceLoss(eps=1e-9, activation='none', beta=1.0, lambda_dice=1.0, lambda_bce=1.0)
optimizer = RAdam(model.parameters(), lr=0.004, weight_decay=0.0001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, cooldown=2,verbose=True)

## 2.2 Training Starts Here

In [ ]:
# number of epochs to train the model
n_epochs = 20#### Due to Paperspace run hour limits

train_loss_list = []
train_dice_list = []
valid_loss_list = []
dice_score_list = []

lr_rate_list = []
valid_loss_min = np.Inf # track change in validation loss
batch_multiplier = 8
model.to(DEVICE)

torch.backends.cudnn.benchmark = True

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    train_dice = 0.0
    dice_score = 0.0
    count = 0  #multiple minibatch
    
    ###################
    # train the model #
    ###################
    model.train()
    bar = tqdm_notebook(train_loader, postfix={"train_loss":0.0,"train_dice":0.0})
    for (images, labels, is_empty) in bar:

        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            images, labels, class_lbls = images.cuda(), labels.cuda(), torch.FloatTensor(is_empty.float()).cuda()
        
        # clear the gradients of all optimized variables
        if count == 0:
            optimizer.step()
            optimizer.zero_grad()
            count = batch_multiplier
        
        # forward pass: compute predicted outputs by passing inputs to the model   
        binary_logits, no_empty_logits, final_logits = model(images)
        
        # calculate the batch loss
        bce_loss_final = criterion(final_logits.reshape(-1), labels.reshape(-1))
        class_loss = nn.BCELoss(reduction='mean')(binary_logits.reshape(-1), class_lbls.reshape(-1))
        
        non_empty = []
        is_empty = class_lbls.reshape(-1)
        for c in range(len(is_empty)):
            if is_empty[c] == 0:
                non_empty.append(c)
                
        has_empty_nonempty = False
        if len(non_empty) * len(is_empty) > 0:
            has_empty_nonempty = True

        all_loss = bce_loss_final + 0.05 * class_loss
        
        if has_empty_nonempty:
            indices = torch.LongTensor(non_empty).cuda()
            y_non_empty = torch.index_select(no_empty_logits.reshape(-1,320,480), 0, indices)
            mask_non_empty = torch.index_select(labels.reshape(-1,320,480), 0, indices)
            loss_no_empty = criterion(y_non_empty.reshape(-1), mask_non_empty.reshape(-1))
            all_loss += 0.50 * loss_no_empty

        all_loss.backward()
        
        train_loss += all_loss.item() * images.size(0)
        dice_cof    = dice_no_threshold(final_logits.reshape(-1).to("cpu"), labels.reshape(-1).to("cpu")).item()
        train_dice += dice_cof * images.size(0)
        
        count -= 1  #multiple minibatch

        bar.set_postfix(ordered_dict={"train_loss":all_loss.item(), "train_dice":dice_cof})
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    
    del images, labels, is_empty

    with torch.no_grad():

        bar = tqdm_notebook(valid_loader, postfix={"valid_loss":0.0, "dice_score":0.0})
        
        for (images, labels, is_empty) in bar:
            
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                images, labels, class_lbls = images.cuda(), labels.cuda(), torch.FloatTensor(is_empty.float()).cuda()
                
            # forward pass: compute predicted outputs by passing inputs to the model
#             binary_logits, no_empty_logits, final_logits = tta.SegmentationTTAWrapper(model, 
#                                                                                       tta.aliases.d4_transform(), 
#                                                                                       merge_mode='mean')(images)
            binary_logits, no_empty_logits, final_logits = model(images)
            
            # calculate the batch loss
            bce_loss_final = criterion(final_logits.reshape(-1), labels.reshape(-1))
            class_loss = nn.BCELoss(reduction='mean')(binary_logits.reshape(-1), class_lbls.reshape(-1))

            non_empty = []
            is_empty = class_lbls.reshape(-1)
            for c in range(len(is_empty)):
                if is_empty[c] == 0:
                    non_empty.append(c)

            has_empty_nonempty = False
            if len(non_empty) * len(is_empty) > 0:
                has_empty_nonempty = True

            all_loss = bce_loss_final + 0.05 * class_loss

            if has_empty_nonempty:
                indices = torch.LongTensor(non_empty).cuda()
                y_non_empty = torch.index_select(no_empty_logits.reshape(-1,320,480), 0, indices)
                mask_non_empty = torch.index_select(labels.reshape(-1,320,480), 0, indices)
                loss_no_empty = criterion(y_non_empty.reshape(-1), mask_non_empty.reshape(-1))
                all_loss += 0.50 * loss_no_empty
                
            # update average validation loss 
            valid_loss += all_loss.item() * images.size(0)
            dice_cof    = dice_no_threshold(final_logits.reshape(-1).to("cpu"), labels.reshape(-1).to("cpu")).item()
            dice_score +=  dice_cof * images.size(0)
            
            bar.set_postfix(ordered_dict={"valid_loss":all_loss.item(), "dice_score":dice_cof})
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    train_dice = train_dice/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    dice_score = dice_score/len(valid_loader.dataset)
    
    train_loss_list.append(train_loss)
    train_dice_list.append(train_dice)
    valid_loss_list.append(valid_loss)
    dice_score_list.append(dice_score)

    lr_rate_list.append([param_group['lr'] for param_group in optimizer.param_groups])
    
    # print training/validation statistics 
    print('Epoch: {} || Training Loss: {:.6f}  Training Dice Score: {:.6f} || Validation Loss: {:.6f} Dice Score: {:.6f}'.format(
          epoch, train_loss, train_dice, valid_loss, dice_score))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), f"{WEIGHT_PATH}/model_{VERSION}'_best.pth'")
        valid_loss_min = valid_loss
    
    scheduler.step(valid_loss)
    

## 2.3 Plot History

In [ ]:
plt.figure(figsize=(10,10))
plt.plot([i[0] for i in lr_rate_list])
plt.ylabel('learing rate during training', fontsize=22)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(train_loss_list,  marker='o', label="Training Loss")
plt.plot(valid_loss_list,  marker='o', label="Validation Loss")
plt.ylabel('loss', fontsize=22)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(dice_score_list)
plt.ylabel('Dice score')
plt.show()

# 3. Predict

In [ ]:
model.load_state_dict(torch.load(f"{WEIGHT_PATH}/model_{VERSION}'_best.pth'"))

model.eval()

In [ ]:
num_workers = N_JOBS
batch_size = 1

valid_dataset = CloudDataset(df=train_df,
                             datatype="valid",
                             img_ids=val_ids,
                             transforms=get_test_augmentation(),)

valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, 
                          num_workers=num_workers)

In [ ]:
import gc
import seaborn as sns

def resize_it(x):
    if x.shape != (350, 525):
        x = cv2.resize(x, dsize=(525, 350), interpolation=cv2.INTER_LINEAR)
    return x

def visualize_with_raw(
    image, mask, original_image=None, original_mask=None, raw_image=None, raw_mask=None
):
    """
    Plot image and masks.
    If two pairs of images and masks are passes, show both.
    """
    fontsize = 14
    class_dict = {0: "Fish", 1: "Flower", 2: "Gravel", 3: "Sugar"}

    f, ax = plt.subplots(3, 5, figsize=(24, 12))

    ax[0, 0].imshow(original_image)
    ax[0, 0].set_title("Original image", fontsize=fontsize)

    for i in range(4):
        ax[0, i + 1].imshow(original_mask[:, :, i])
        ax[0, i + 1].set_title(f"Original mask {class_dict[i]}", fontsize=fontsize)

    ax[1, 0].imshow(raw_image)
    ax[1, 0].set_title("Original image", fontsize=fontsize)

    for i in range(4):
        ax[1, i + 1].imshow(raw_mask[:, :, i])
        ax[1, i + 1].set_title(f"Raw predicted mask {class_dict[i]}", fontsize=fontsize)

    ax[2, 0].imshow(image)
    ax[2, 0].set_title("Transformed image", fontsize=fontsize)

    for i in range(4):
        ax[2, i + 1].imshow(mask[:, :, i])
        ax[2, i + 1].set_title(
            f"Predicted mask with processing {class_dict[i]}", fontsize=fontsize
        )
        
def mask2rle(img):
    """
    Convert mask to rle.
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    """
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)


In [ ]:
def post_process(probability, threshold, min_size):
    """
    This is slightly different from other kernels as we draw convex hull here itself.
    Post processing of each predicted mask, components with lesser number of pixels
    than `min_size` are ignored
    """
    mask = (cv2.threshold(probability, threshold, 1, cv2.THRESH_BINARY)[1])
    mask = draw_convex_hull(mask.astype(np.uint8))
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
    predictions = np.zeros((350, 525), np.float32)
    num = 0
    for c in range(1, num_component):
        p = component == c
        if p.sum() > min_size:
            predictions[p] = 1
            num += 1
    return predictions, num

def draw_convex_hull(mask, mode='convex'):
    
    img = np.zeros(mask.shape)
    contours, hier = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        if mode=='rect': # simple rectangle
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 255, 255), -1)
        if mode=='convex': # minimum convex hull
            hull = cv2.convexHull(c)
            cv2.drawContours(img, [hull], 0, (255, 255, 255),-1)
        else: # minimum area rectangle
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(img, [box], 0, (255, 255, 255),-1)
    return img/255.

In [ ]:
valid_masks = []
count = 0
tr = min(len(val_ids)*4, 2220)
probabilities = np.zeros((tr, 350, 525), dtype = np.float32)
for data, target, _ in tqdm_notebook(valid_loader):
    if train_on_gpu:
        data = data.cuda()
    target = target.cpu().detach().numpy()
    _,_,pred_masks = model(data)
    outpu = pred_masks.cpu().detach().numpy()
    for p in range(data.shape[0]):
        output, mask = outpu[p], target[p]
        for m in mask:
            valid_masks.append(resize_it(m))
        for probability in output:
            probabilities[count, :, :] = resize_it(probability)
            count += 1
        if count >= tr - 1:
            break
    if count >= tr - 1:
        break

In [ ]:
class_params = {}
dice_scores = []
for class_id in range(4):
    print('-----------------------')
    print([k for k, v in class_names_dict.items() if v == class_id + 1][0])
    attempts = []
    for t in tqdm_notebook(range(0, 100, 5)):
        t /= 100
        for ms in [5000, 10000, 15000, 20000, 30000]:
            masks, d = [], []
            for i in range(class_id, len(probabilities), 4):
                probability = probabilities[i]
                predict, num_predict = post_process(probability, t, ms)
                masks.append(predict)
            for i, j in zip(masks, valid_masks[class_id::4]):
                if (i.sum() == 0) & (j.sum() == 0):
                    d.append(1)
                else:
                    d.append(dice_coef_np(i, j))
            attempts.append((t, ms, np.mean(d)))

    attempts_df = pd.DataFrame(attempts, columns=['threshold', 'size', 'dice'])
    attempts_df = attempts_df.sort_values('dice', ascending=False)
    print(attempts_df.head())
    best_threshold = attempts_df['threshold'].values[0]
    best_size = attempts_df['size'].values[0]
    dice_scores.append(attempts_df['dice'].values[0])
    class_params[class_id] = (best_threshold, best_size)
    
print('-----------------------')
print(class_params)
print('-----------------------')
print('Validation Score : ', np.mean(dice_scores))


In [ ]:
for i, (data, target, _) in enumerate(valid_loader):
    if train_on_gpu:
        data = data.cuda()
    _,_,pred_masks = model(data)
    output = ((pred_masks)[0]).cpu().detach().numpy()
    image  = data[0].cpu().detach().numpy()
    mask   = target[0].cpu().detach().numpy()
    output = output.transpose(1 ,2, 0)
    image_vis = image.transpose(1, 2, 0)
    mask = mask.astype('uint8').transpose(1, 2, 0)
    pr_mask = np.zeros((350, 525, 4))
    for j in range(4):
        probability = resize_it(output[:, :, j])
        pr_mask[:, :, j], _ = post_process(probability,
                                           class_params[j][0],
                                           class_params[j][1])
    visualize_with_raw(image=image_vis, mask=pr_mask,
                      original_image=image_vis, original_mask=mask,
                      raw_image=image_vis, raw_mask=output)
    if i >= 6:
        break

# 4. Inference

In [ ]:
batch_size = 1
test_dataset = CloudDataset(df=sub_df,
                            datatype='test', 
                            img_ids=test_ids,
                            transforms=get_test_augmentation())

test_loader = DataLoader(test_dataset, batch_size=batch_size,
                         shuffle=False, num_workers=N_JOBS)

In [ ]:
subm = pd.read_csv("../input/sample_submission.csv")
pathlist = [TEST_PATH+'/' + i.split("_")[0] for i in subm['Image_Label']]

In [ ]:
def get_black_mask(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (525,350))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 0, 0], np.uint8)
    upper = np.array([180, 255, 10], np.uint8)
    return (~ (cv2.inRange(hsv, lower, upper) > 250)).astype(int)

plt.imshow(get_black_mask(pathlist[120]))
plt.show()

In [ ]:
encoded_pixels = []
image_id = 0
cou = 0
np_saved = 0
for data, target, _ in tqdm_notebook(test_loader):
    if train_on_gpu:
        data = data.cuda()
    _,_,output = model(data)
    del data
    for i, batch in enumerate(output):
        for probability in batch:
            probability = resize_it(probability.cpu().detach().numpy())
            predict, num_predict = post_process(probability,
                                                class_params[image_id % 4][0],
                                                class_params[image_id % 4][1])
            if num_predict == 0:
                encoded_pixels.append('')
            else:
                black_mask = get_black_mask(pathlist[cou])
                np_saved += np.sum(predict)
                predict = np.multiply(predict, black_mask)
                np_saved -= np.sum(predict)
                r = mask2rle(predict)
                encoded_pixels.append(r)
            cou += 1
            image_id += 1

print(f"number of pixel saved {np_saved}")

# 5. Submission

In [ ]:
sub_df['EncodedPixels'] = encoded_pixels
sub_df.to_csv(SUBMISSION_PATH+f'/submission_{VERSION}.csv', columns=['Image_Label', 'EncodedPixels'], index=False)
sub_df.head(10)